# RareSkills Zero Knowledge Week 2

## Modular arithmetic essentials
### Benchmarks
Here is how you do modular arithmetic in python. Run the two cells below and note how different their execution times are. You should use `pow` instead of doing modular arithmetic with the elementary operators.

In [1]:
import time

s = time.time()
ans = 1002583 ** 939001 % 2003951
t = time.time()

print("calculating {} took {:.2f} seconds".format(ans, t - s))

calculating 1590868 took 6.25 seconds


In [2]:
s = time.time()
ans = pow(1002583,939001,2003951)
t = time.time()

print("calculating {} operation took {:.2f} seconds".format(ans, t - s))

calculating 1590868 operation took 0.00 seconds


### Encrypting a number

If we start with a generator `g` and raise it to a power `s` mod `n`, then in general we cannot determine `s` even if `g` and `n` are known. This is the discrete logarithm problem.

We can say we "encrypt" `s` using the scheme

$$ E(s) = g^{s} \pmod n $$

But this means we need to choose `g` and `n` carefully. If `g` is 3, `s` is 2, and `n` is a large number, we can intelligently guess what `s` is in this case. We want to pick `g` and `n` such that all members "wrap around." We won't concern ourselves with this here, but you should be aware of it. The other issue is not picking a sufficiently large `n`. 

### Computing the discrete logarithm
If we do `s_p = g^s mod n`, and make `g` and `n` public, then it is infeasible to compute `s`. But this is only true if `n` is sufficiently large.

In the following example, we provide `s_p`, `g`, and `n`. Use python to bruteforce the solution to find `s`.

$$g^{ac} = (g^{a})^{c} \pmod p$$

In [3]:
n = 9551
g = 5
encrypted_number = 5666

<font color='red'>**Assignment 1: Use python to bruteforce the solution to find `s`**</font>

In [5]:
# Your Solution and Answer Here
def calc_pow(g, n, v):
    for s in range(n):
        if pow(g, s, n) == v:
            return s
    return None

student_solution = calc_pow(g, n, encrypted_number); print(student_solution)

2531


In [6]:
assert pow(g, student_solution, n) == encrypted_number
print("student_solution is {}".format(student_solution))

student_solution is 2531


### Computing the discrete logarithm the smart way
The smart way to do optimal algorithms is to [copy and paste from stackoverflow](https://stackoverflow.com/a/58665206/2079806). You do not have to understand how the library works

<font color='red'>**Assignment 2: Break the cryptography below and obtain the secret number**</font>

In [7]:
n = 1000004119
g = 5
encrypted_number = 767805982

In [10]:
# Your Solution and Answer Here
from sympy.ntheory import discrete_log
student_solution = discrete_log(n, encrypted_number, g)
student_solution

420

In [11]:
assert pow(g, student_solution, n) == encrypted_number
print("student_solution is {}".format(student_solution))

student_solution is 420


## How Zero Knowledge Addition Works

A Zero Knowledge Proof verifies a computation is true without revealing a the inputs to the computation.

That is, if a + b = c, then we want a "trapdoor function" E(x) such that E(a)■E(b) = E(c) where ■ is the binary operator of the group E(x) is an element of.

So to prove a + b = c without revealing a, b, or c, we apply A = E(a), B = E(b), C = E(c) and give (A, B, C) to a verifier.

The verifier will then check A ■ B == C.

Although they don't know the values A, B, and C "came from", they do know the unencrypted values form a valid sum.

## Zero Knowledge Addition
The following property is very important. It lets us verify the addition of numbers (integers under addition) using another group: $g^x \pmod p$ under multiplication. Because of the discrete logarithm, we cannot easily go from $g^x \pmod p$ to recover x.

a + b = c is homomorphic to $g^ag^b = g^c \pmod p$.

$$ g^{a}g^{b} = g^{a + b} \pmod n $$

Try a few different values of `a` and `b` yourself to see it in action

In [15]:
import math
n = 1000004119
g = 5
a = math.floor(n/3 - 1)
b = n - 5
print(pow(g, a, n) * pow(g, b, n) == pow(g, a + b, n))

False


Wait what?! That's supposed to be true?

<font color='red'>**Assignment 3: The code above has a bug. What is the bug?**</font>

In [16]:
# Your corrected code here
print((pow(g, a, n) * pow(g, b, n)) % n == pow(g, a + b, n))

True


## Zero Knowledge Subtraction
We can also encrypt the operation a - b. This is the same as 

$$ g^{a}g^{-b} = g^{a-b} $$

But we can't just stick a negative sign in front of the exponent

In [17]:
g = 57
n = 101
g ** -5 % n

1.6619797259514097e-09

The above is not an integer! However, python is smart enough to know what you are doing if you use `pow`. To test this, we expect

$$ g^{-a}g^{a} = 1 $$

because g^0 is 1, and `a - a = 0`.

In [18]:
a = 22
pow(g, a, n) * pow(g, -a, n) % n == 1

True

So what magic is happening behind the scenes? The formula below is used to compute the modular inverse.

$$ a^{-1} = a^{n - 2} \pmod n $$

In [19]:
a_inv = a ** (n - 2) % n
a_inv * a % n == 1

True

## Multiplication by a constant
Multiplication by a constant is really just repeated addition

$$ (g^{a})^{4} = g^{a} g^{a} g^{a} g^{a} $$

This allows us to "multiply by a constant" in a zero knowledge fashion.

In [20]:
a = 15
pow(g, a, n) * pow(g, a, n) * pow(g, a, n) * pow(g, a, n) % n == pow(g, a * 4, n)

True

Of course, it would be annoying to multiply like that if you have a big coefficient, so the following is better

In [21]:
pow(pow(g, a, n), 4 , n) == pow(g, a * 4, n)

True

## I know the solution to a systems of equations

Now you should be able to prove to me that you know the solution to in a zero knowledge fashion

$$
2x + 8y = 7944\\
5x + 3y = 4764
$$

<font color='red'>**Assignment 4: conver the above equation to a zk proof system where you can prove to a verifier (with an agreed upon g and n) that you know the solution to an agreed upon system of equation, without revealing the solution**</font>

```python
g = 57
n = 1000004119
```

The system of equations I give to verifier is:

$$ (g^{x})^{2}(g^{y})^{8} = g^{4764}\ mod\ n $$

$$ (g^{x})^{79445}(g^{y})^{3} = g^{4764}\ mod\ n $$

Instead of giving $x$ and $y$ to the verifier, I will pass $g^{x}$ and $g^{y}$ to him saying these values will satisfy the above equations.

In [56]:
g = 57
n = 1000004119

import numpy as np
import galois
GF = galois.GF(n)
A = GF([[2, 8], [79445, 3]])
b = GF([4764, 4764])
[x, y] = np.linalg.solve(A, b)
print(f"secret x: {x}, secret y: {y}")

# verify
assert GF(2) * x + GF(8) * y == GF(4764)
assert GF(79445) * x + GF(3) * y == GF(4764)

# Instead of passing x and y to the verifier, I will pass g_x and g_y
g_x = np.power(GF(g), int(x))
g_y = np.power(GF(g), int(y))
assert g_x == pow(g, int(x), n)
print(f"g_x: {g_x}, g_y: {g_y}")

# verifier verifies the 1st equation
lhs = np.power(g_x, 2) * np.power(g_y, 8); print(f"lhs: {lhs}")
rhs = np.power(GF(g), 4764); print(f"rhs: {rhs}")
assert lhs == rhs

secret x: 261099267, secret y: 684728868
g_x: 32248980, g_y: 331957607
lhs: 850956665
rhs: 366446438


AssertionError: 

## Convert a rational number to a finite field element

<font color='red'>**Assignment 5: Compute 53/192 + 61/511 (mod 1033) in python. Show your work. You are not required to implement euclid's algorithm. Show it is equal to the original rational number sum: https://www.wolframalpha.com/input?i=53%2F192+%2B+61%2F511**</font>

In [57]:
p = 1033
l1 = (53 * pow(192, -1, p)) % p; print(f"l1: {l1}")
l2 = (61 * pow(511, -1, p)) % p; print(f"l2: {l2}")
res = (l1 + l2) % p; print(res)

# 53/192 + 61/511
num = (53 * 511 + 61 * 192); print(f"num: {num}")
denom = (192 * 511); print(f"denom: {denom}")
assert (num * pow(denom, -1, p)) % p == res

l1: 619
l2: 928
514
num: 38795
denom: 98112
